In [ ]:
import os
import csv
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

network = Network("./porto/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

## Precompute an UBODT table

# Can be skipped if you already generated an ubodt file
ubodt_gen = UBODTGenAlgorithm(network,graph)
status = ubodt_gen.generate_ubodt("./porto/ubodt.txt", 0.02, binary=False, use_omp=True)      ###threshold of 2km
print(status)

### Read UBODT
ubodt = UBODT.read_ubodt_csv("./porto/ubodt.txt")
### Create FMM model
model = FastMapMatch(network,graph,ubodt)
k=6
radius=0.05
gps_error=0.0002
fmm_config = FastMapMatchConfig(k,radius,gps_error)

from tqdm import tqdm
with open("./data/improved_train_1000.csv","r") as csvfile:
    reader = csv.reader(csvfile)    
    train1000 = [row[9] for row in reader][1:]

with open("./data/matched_routines.csv","w") as csvfile:
    output = csv.writer(csvfile)
    output.writerow(["Index", "cpath","mgeom", "opath", "offset", "length", "spdist"])
    for index in tqdm(range(len(train1000))):
        gps = eval(train1000[index])
        wkt = 'LINESTRING('+','.join([' '.join([str(j) for j in i]) for i in gps])+')'
        output = model.match_wkt(wkt, fmm_config)

        candidates = list(output.candidates)
        output.writerow([index, str(list(output.cpath)), output.mgeom.export_wkt(), str(list(output.opath)), str([c.offset for c in candidates]), str([c.length for c in candidates]), str([c.spdist for c in candidates])])